In [1]:
import pandas as pd

In [99]:
ratings_movies = pd.read_csv('data/ratings_movies.csv')

ratings_movies.head(2)

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance


In [10]:
import re

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [30]:
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies['year_release'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 100836 entries, 0 to 100835
Series name: year_release
Non-Null Count   Dtype  
--------------   -----  
100818 non-null  float64
dtypes: float64(1)
memory usage: 787.9 KB


In [49]:
a = ratings_movies[ratings_movies['year_release'] == 1999].groupby(by='title')['rating'].mean()
a.sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [53]:
ratings_movies[ratings_movies['year_release'] == 2010].groupby(by='genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [57]:
ratings_movies.groupby(by='userId')['genres'].nunique().nlargest()

userId
599    524
414    482
448    403
380    399
474    395
Name: genres, dtype: int64

In [88]:
# Найдите пользователя, который выставил наименьшее количество оценок, 
# но его средняя оценка фильмам наибольшая.
ratings_movies.groupby(by='userId')['rating'].agg(['count', 'mean']).sort_values(
    by=['count', 'mean'], 
    ascending=[True, False]
    )


,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [97]:
mask1 = ratings_movies['year_release'] == 2018
a = ratings_movies[mask1].groupby(by='genres')['rating'].agg(['mean', 'count'])

mask2 = a['count'] > 10
a[mask2].sort_values(by='count', ascending=False)


,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [144]:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year

a = ratings_movies.pivot_table(
    values='rating', 
    index='year_rating',
    columns='genres',
    aggfunc='mean'
)

In [159]:
orders = pd.read_csv('data/orders.csv', sep=';')
products = pd.read_csv('data/products.csv', sep=';')

display(orders.head(2))
display(products.head(2))

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR


In [203]:
orders_products = orders.merge(right=products, how='left', left_on='ID товара', right_on='Product_ID')

orders_products['profit'] = orders_products['Price'] * orders_products['Количество']
mask = orders_products['Оплачен'] == 'Да'
orders_products[mask].groupby('ID Покупателя')['profit'].sum().sort_values()

ID Покупателя
1        1
8     1344
5    13043
7    17096
Name: profit, dtype: int64